# Prompt Manager End-to-End Test

This notebook provides a comprehensive test of the prompt manager functionality:
1. Load prompts from YAML
2. Render prompts with variables
3. Make Anthropic API calls
4. Validate responses

**Prerequisites:**
- Add your Anthropic API key to `.env` file in the project root
- Install required dependencies: `pip install anthropic python-dotenv`

## 1. Imports

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import anthropic
import json

In [2]:
# Add src to path to import prompt_manager
project_root = Path.cwd().parent
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

from prompt_manager import PromptManager

print("✓ Imported PromptManager")

✓ Imported PromptManager


## 2. Load Environment Variables

In [3]:
# Load .env file from project root
env_path = project_root / ".env"
load_dotenv(env_path)

# Verify API key is loaded
api_key = os.getenv("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key loaded (length: {len(api_key)})")
else:
    print("✗ API key not found in .env file")
    print(f"  Expected location: {env_path}")

✓ API key loaded (length: 108)


In [ ]:
# Helper for environments without top-level await support
import asyncio
import nest_asyncio

# Allow nested event loops (for Jupyter)
nest_asyncio.apply()

def run_async(coro):
    """Helper to run async code in sync context."""
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro)

print("✓ Async helper configured")

## 3. Initialize Prompt Manager

In [4]:
# Create prompt manager using simplified API
# This automatically creates storage and registry with sensible defaults
manager = run_async(PromptManager.create(
    prompt_dir="./prompts",  # Directory for storing prompts
    auto_load_yaml=True,      # Automatically load YAML files
))

print("✓ Prompt manager initialized with simplified API")

✓ Prompt manager initialized with simplified API


## 4. Load Prompts from YAML

In [8]:
# Verify YAML prompts were auto-loaded
prompts = run_async(manager.list_prompts())
print(f"✓ {len(prompts)} prompts loaded automatically from YAML files")

# Show what was loaded
for prompt in prompts:
    print(f"  - {prompt.id} v{prompt.version}")

AttributeError: 'coroutine' object has no attribute 'list_prompts'

## 5. List Available Prompts

In [6]:
# List all prompts
prompts = run_async(manager.list_prompts())

#print(f"Available prompts ({len(prompts)}):")  
print()
for prompt in prompts:
    print(f"ID: {prompt.id}")
    print(f"  Version: {prompt.version}")
    print(f"  Format: {prompt.format.value}")
    print(f"  Status: {prompt.status.value}")
    print(f"  Description: {prompt.metadata.description if prompt.metadata else 'N/A'}")
    print()


AttributeError: 'coroutine' object has no attribute 'list_prompts'

## 6. Render a Text Prompt

In [ ]:
# Render a simple text prompt
prompt_id = "greeting"  # Adjust based on available prompts
variables = {
    "name": "Alice",
    "language": "English"
}

rendered = manager.render(prompt_id, variables=variables)

print(f"Rendered prompt '{prompt_id}':")
print("-" * 50)
print(rendered)
print("-" * 50)

## 7. Test Anthropic Integration - Text Prompt

In [ ]:
# Initialize Anthropic client
client = anthropic.Anthropic(api_key=api_key)

# Get the prompt using manager
prompt = run_async(manager.get_prompt("greeting"))

# Create integration
integration = AnthropicIntegration(template_engine=manager._template_engine)

# Convert prompt to Anthropic format
messages = integration.convert(prompt, variables=variables)

print("Converted messages for Anthropic:")
print(json.dumps(messages, indent=2))


In [ ]:
# Make API call
response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=messages
)

print("Response from Anthropic:")
print("-" * 50)
print(response.content[0].text)
print("-" * 50)
print(f"\nTokens used: {response.usage.input_tokens} input, {response.usage.output_tokens} output")

## 8. Test Chat Prompt

In [ ]:
# Try a chat format prompt if available
all_prompts = run_async(manager.list_prompts())
chat_prompts = [p for p in all_prompts if p.format.value == "chat"]

if chat_prompts:
    chat_prompt_id = chat_prompts[0].id
    print(f"Testing chat prompt: {chat_prompt_id}")
    
    # Adjust variables based on the prompt
    chat_variables = {
        "user_message": "I need help with my order #12345",
        "user_name": "Bob"
    }
    
    # Get and convert prompt
    chat_prompt = run_async(manager.get_prompt(chat_prompt_id))
    chat_messages = integration.convert(chat_prompt, variables=chat_variables)
    
    print("\nChat messages:")
    print(json.dumps(chat_messages, indent=2))
else:
    print("No chat prompts found")


In [ ]:
# Make API call with chat prompt
if chat_prompts:
    chat_response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        messages=chat_messages
    )
    
    print("Chat response from Anthropic:")
    print("-" * 50)
    print(chat_response.content[0].text)
    print("-" * 50)
    print(f"\nTokens used: {chat_response.usage.input_tokens} input, {chat_response.usage.output_tokens} output")

## 9. Test Schema Validation

In [ ]:
# Create a test prompt with output schema
from prompt_manager.core.models import Prompt, PromptFormat, PromptStatus, PromptTemplate

# Create a simple extraction prompt
extraction_prompt = Prompt(
    id="extract_info",
    version="1.0.0",
    format=PromptFormat.TEXT,
    status=PromptStatus.ACTIVE,
    template=PromptTemplate(
        content="Extract the name and email from this text: {{text}}\n\nReturn as JSON with 'name' and 'email' fields.",
        variables=["text"]
    )
)

# Register the prompt
manager.create_prompt(extraction_prompt)
print("✓ Created extraction prompt")

In [ ]:
# Render and call Anthropic
extraction_vars = {
    "text": "Contact John Smith at john.smith@example.com for more information."
}

extraction_prompt = run_async(manager.get_prompt("extract_info"))
extraction_messages = integration.convert(
    extraction_prompt,
    variables=extraction_vars
)

extraction_response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=256,
    messages=extraction_messages
)

result_text = extraction_response.content[0].text
print("Extraction result:")
print(result_text)


In [ ]:
# Parse and validate JSON response
try:
    # Extract JSON from response (handle markdown code blocks)
    import re
    
    # Try to find JSON in code blocks first
    json_match = re.search(r'```json\s*(.+?)\s*```', result_text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
    else:
        # Try to find raw JSON
        json_match = re.search(r'\{.+\}', result_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
        else:
            json_str = result_text
    
    parsed_result = json.loads(json_str)
    
    print("✓ Successfully parsed JSON:")
    print(json.dumps(parsed_result, indent=2))
    
    # Validate structure
    required_fields = ["name", "email"]
    missing_fields = [f for f in required_fields if f not in parsed_result]
    
    if missing_fields:
        print(f"\n✗ Missing required fields: {missing_fields}")
    else:
        print("\n✓ All required fields present")
        print(f"  Name: {parsed_result['name']}")
        print(f"  Email: {parsed_result['email']}")
        
except json.JSONDecodeError as e:
    print(f"✗ Failed to parse JSON: {e}")
    print(f"  Response text: {result_text}")

## 10. Test Summary

In [ ]:
print("=" * 60)
print("PROMPT MANAGER TEST SUMMARY")
print("=" * 60)
print()
print("✓ Environment setup")
print("✓ Prompt manager initialized")
print(f"✓ Loaded {count} prompts from YAML")
print("✓ Rendered text prompt")
print("✓ Anthropic integration working")
print("✓ Text prompt API call successful")
if chat_prompts:
    print("✓ Chat prompt API call successful")
print("✓ JSON extraction and validation working")
print()
print("All tests passed!")
print("=" * 60)